In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

In [2]:
#import file 
yelp_file = "Resources/yelp_business.csv"
yelp_df = pd.read_csv(yelp_file)
yelp_df.head(3)

,business_id,name,neighborhood,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,categories
0,FYWN1wneV18bWNgQjJ2GNg,"""Dental by Design""",NaN,"""4855 E Warner Rd, Ste B9""",Ahwatukee,AZ,85044,33.330690,-111.978599,4.0,22,1,Dentists;General Dentistry;Health & Medical;Or...
1,He-G7vWjzVUysIKrfNbPUQ,"""Stephen Szabo Salon""",NaN,"""3101 Washington Rd""",McMurray,PA,15317,40.291685,-80.104900,3.0,11,1,Hair Stylists;Hair Salons;Men's Hair Salons;Bl...
2,KQPW8lFf1y5BT2MxiSZ3QA,"""Western Motor Vehicle""",NaN,"""6025 N 27th Ave, Ste 1""",Phoenix,AZ,85017,33.524903,-112.115310,1.5,18,1,Departments of Motor Vehicles;Public Services ...


In [3]:
# Create a filter DF with only needed columns and rename to import in sql later
yelp_cols = ["name", "address", "city", "state", "postal_code", "stars", "review_count", "categories"]
yelp_df2 = yelp_df[yelp_cols].copy()

# Rename the column headers
yelp_df2 = yelp_df2.rename(columns={"state": "state_cd",
                                    "name":"biz_name",

                                             
                                               })
yelp_df2.head(3)

,biz_name,address,city,state_cd,postal_code,stars,review_count,categories
0,"""Dental by Design""","""4855 E Warner Rd, Ste B9""",Ahwatukee,AZ,85044,4.0,22,Dentists;General Dentistry;Health & Medical;Or...
1,"""Stephen Szabo Salon""","""3101 Washington Rd""",McMurray,PA,15317,3.0,11,Hair Stylists;Hair Salons;Men's Hair Salons;Bl...
2,"""Western Motor Vehicle""","""6025 N 27th Ave, Ste 1""",Phoenix,AZ,85017,1.5,18,Departments of Motor Vehicles;Public Services ...


In [4]:
#CLEAN UP -  remove "" from columns name and address #df.Geo.str.replace('"','') 

yelp_df2["biz_name"] = yelp_df2.biz_name.str.replace('"','')

yelp_df2["address"] = yelp_df2.address.str.replace('"','')

# remove "," from columns address
yelp_df2["address"] = yelp_df2.address.str.replace(',',' ')

# set name & address to lower case
yelp_df2["biz_name"] = yelp_df2["biz_name"].str.lower()
yelp_df2["address"] = yelp_df2["address"].str.lower()


yelp_df2.head(3)

,biz_name,address,city,state_cd,postal_code,stars,review_count,categories
0,dental by design,4855 e warner rd ste b9,Ahwatukee,AZ,85044,4.0,22,Dentists;General Dentistry;Health & Medical;Or...
1,stephen szabo salon,3101 washington rd,McMurray,PA,15317,3.0,11,Hair Stylists;Hair Salons;Men's Hair Salons;Bl...
2,western motor vehicle,6025 n 27th ave ste 1,Phoenix,AZ,85017,1.5,18,Departments of Motor Vehicles;Public Services ...


In [5]:
#Filter to just NV 
yelp_nv = yelp_df2[yelp_df2["state_cd"]== "NV"]
#drop null
yelp_nv = yelp_nv.dropna(subset=["postal_code"])
yelp_nv

,biz_name,address,city,state_cd,postal_code,stars,review_count,categories
6,bdj realty,2620 regatta dr ste 102,Las Vegas,NV,89128,4.0,5,Real Estate Services;Real Estate;Home Services...
7,soccer zone,7240 w lake mead blvd ste 4,Las Vegas,NV,89128,1.5,9,Shopping;Sporting Goods
9,detailing gone mobile,,Henderson,NV,89014,5.0,7,Automotive;Auto Detailing
26,alfredo's jewelry,5775 s eastern ste 103,Las Vegas,NV,89119,4.5,23,Shopping;Jewelry;Watch Repair;Local Services
37,pampered hair passionate about hair,2470 paseo verde pkwy,Henderson,NV,89074,5.0,3,Hair Salons;Blow Dry/Out Services;Hair Stylist...
...,...,...,...,...,...,...,...,...
174539,stonegate real estate services,3030 s jones blvd ste 105,Las Vegas,NV,89146,4.5,15,Real Estate Services;Home Services;Property Ma...
174545,starbucks,591 n eastern ave ste 110,Las Vegas,NV,89101,2.5,35,Coffee & Tea;Food
174546,sprint store,7325 s rainbow blvd ste 120,Las Vegas,NV,89139,4.0,35,Mobile Phone Accessories;Mobile Phones;Electro...
174552,9 dragons fight shop,3375 s decatur blvd ste 7,Las Vegas,NV,89102,5.0,5,Martial Arts;Sports Wear;Men's Clothing;Sporti...


In [6]:
# strip zip codes numbers after -
yelp_nv["postal_code"] = yelp_nv["postal_code"].str.rstrip("-")

#drop rows with no values in columns
yelp_nv = yelp_nv.loc[yelp_nv["address"]!= ""]
yelp_nv = yelp_nv.loc[yelp_nv["biz_name"]!= ""]
yelp_nv = yelp_nv.loc[yelp_nv["city"]!= ""]
yelp_nv = yelp_nv.loc[yelp_nv["state_cd"]!= ""]
yelp_nv = yelp_nv.loc[yelp_nv["postal_code"]!= ""]
yelp_nv = yelp_nv.loc[yelp_nv["stars"]!= ""]
yelp_nv = yelp_nv.loc[yelp_nv["review_count"]!= ""]
yelp_nv = yelp_nv.loc[yelp_nv["categories"]!= ""]

yelp_nv

C:\Users\j68im\Anaconda3\envs\PythonData\lib\site-packages\pandas\core\ops\array_ops.py:252: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


,biz_name,address,city,state_cd,postal_code,stars,review_count,categories
6,bdj realty,2620 regatta dr ste 102,Las Vegas,NV,89128,4.0,5,Real Estate Services;Real Estate;Home Services...
7,soccer zone,7240 w lake mead blvd ste 4,Las Vegas,NV,89128,1.5,9,Shopping;Sporting Goods
26,alfredo's jewelry,5775 s eastern ste 103,Las Vegas,NV,89119,4.5,23,Shopping;Jewelry;Watch Repair;Local Services
37,pampered hair passionate about hair,2470 paseo verde pkwy,Henderson,NV,89074,5.0,3,Hair Salons;Blow Dry/Out Services;Hair Stylist...
41,task electric,7260 cimarron rd ste 130,Las Vegas,NV,89113,4.5,20,Home Services;Lighting Fixtures & Equipment;Lo...
...,...,...,...,...,...,...,...,...
174539,stonegate real estate services,3030 s jones blvd ste 105,Las Vegas,NV,89146,4.5,15,Real Estate Services;Home Services;Property Ma...
174545,starbucks,591 n eastern ave ste 110,Las Vegas,NV,89101,2.5,35,Coffee & Tea;Food
174546,sprint store,7325 s rainbow blvd ste 120,Las Vegas,NV,89139,4.0,35,Mobile Phone Accessories;Mobile Phones;Electro...
174552,9 dragons fight shop,3375 s decatur blvd ste 7,Las Vegas,NV,89102,5.0,5,Martial Arts;Sports Wear;Men's Clothing;Sporti...


In [7]:
#create new column in new data frame with address and name to use as key
yelp_transformed = yelp_nv

yelp_transformed["name_key"] = yelp_transformed["biz_name"] + " " + yelp_transformed["address"] 
yelp_transformed.head(3)

,biz_name,address,city,state_cd,postal_code,stars,review_count,categories,name_key
6,bdj realty,2620 regatta dr ste 102,Las Vegas,NV,89128,4.0,5,Real Estate Services;Real Estate;Home Services...,bdj realty 2620 regatta dr ste 102
7,soccer zone,7240 w lake mead blvd ste 4,Las Vegas,NV,89128,1.5,9,Shopping;Sporting Goods,soccer zone 7240 w lake mead blvd ste 4
26,alfredo's jewelry,5775 s eastern ste 103,Las Vegas,NV,89119,4.5,23,Shopping;Jewelry;Watch Repair;Local Services,alfredo's jewelry 5775 s eastern ste 103


In [8]:
#import file
ri_file = "Resources/restaurant_inspections.csv"
ri_df = pd.read_csv(ri_file)
ri_df.head()

,Serial Number,Permit Number,Restaurant Name,Location Name,Category Name,Address,City,State,Zip,Current Demerits,...,Inspection Time,Employee ID,Inspection Type,Inspection Demerits,Inspection Grade,Permit Status,Inspection Result,Violations,Record Updated,Location 1
0,DA0BEBWGS,PR0003177,7-Eleven #32224 - SNACK BAR,7-Eleven Convenience Store #32224,Snack Bar,5576 Boulder Hwy,Las Vegas,Nevada,89122-6010,3,...,02/09/2017 10:00:00 AM,EE7000559,Routine Inspection,3,A,A,'A' Grade,"215,2927",02/09/2017 10:37:14 AM,"(36.101164, 115.0522322)"
1,DA003LUQ4,PR0009484,VENETIAN MAIN KITCHEN WAREWASH,VENETIAN HOTEL & CASINO,Special Kitchen,3355 S Las Vegas Blvd,Las Vegas,Nevada,89109-8941,0,...,10/24/2017 01:30:00 PM,EE7001166,Routine Inspection,0,A,A,'A' Grade,NaN,10/24/2017 02:51:05 PM,"(36.1226902, 115.1701939)"
2,DA00DC1UZ,PR0112115,PIZZA ROCK - COOKLINE / PANTRY,PIZZA ROCK,Special Kitchen,2300 PASEO VERDE Pkwy,Henderson,Nevada,89052,5,...,10/04/2017 02:20:00 PM,EE7001125,Routine Inspection,5,A,A,'A' Grade,2954,10/06/2017 09:22:08 AM,"(36.0205397, 115.0895425)"
3,DA02IDZBM,PR0116003,BELL'S BBQ,BELL'S BBQ,Restaurant,10895 S EASTERN 130,Henderson,Nevada,89052,0,...,10/23/2017 04:30:00 PM,EE7001207,Routine Inspection,20,B,B,'B' Downgrade,"209,211,212,215,2908,2909,2910,2930",10/24/2017 10:34:05 AM,"(35.993777, 115.101582)"
4,DA07ZFDM0,PR0020713,Dairy Queen,DAIRY QUEEN,Restaurant,7501 W LAKE MEAD,Las Vegas,Nevada,89128,8,...,10/10/2017 03:15:00 PM,EE7001215,Routine Inspection,8,A,A,'A' Grade,"202,2909,2930",10/10/2017 03:54:53 PM,"(36.19518, 115.2573615)"


In [9]:
# Create a filtered dataframe from specific columns # keep:“permit code”,“Restaurant Name”, “address”, “city”, “state”, “zip”, “inspection time”, “Inspection results”, “location”
ri_cols = ["Permit Number","Restaurant Name", "Address", "City", "State", "Zip", "Inspection Time", "Inspection Result"]
ri_transformed = ri_df[ri_cols].copy()

# Rename the column headers
ri_transformed = ri_transformed.rename(columns={"Permit Number": "permit_number",
                                                "Restaurant Name":"restaurant_name",
                                                "Address": "address",
                                                "State":"state_cd",
                                                "Zip": "zip",
                                                "City": "city",
                                                "Inspection Time":"inspection_time",
                                                "Inspection Result":"inspection_result"
                                                
                                               })

ri_transformed.head(3)

,permit_number,restaurant_name,address,city,state_cd,zip,inspection_time,inspection_result
0,PR0003177,7-Eleven #32224 - SNACK BAR,5576 Boulder Hwy,Las Vegas,Nevada,89122-6010,02/09/2017 10:00:00 AM,'A' Grade
1,PR0009484,VENETIAN MAIN KITCHEN WAREWASH,3355 S Las Vegas Blvd,Las Vegas,Nevada,89109-8941,10/24/2017 01:30:00 PM,'A' Grade
2,PR0112115,PIZZA ROCK - COOKLINE / PANTRY,2300 PASEO VERDE Pkwy,Henderson,Nevada,89052,10/04/2017 02:20:00 PM,'A' Grade


In [19]:
#shorten zip code 
ri_transformed["zip"] = ri_transformed["zip"].str.rstrip("-")
ri_transformed

,permit_number,restaurant_name,address,city,state_cd,zip,inspection_time,inspection_result,name_key
0,PR0003177,7-eleven #32224 - snack bar,5576 boulder hwy,Las Vegas,Nevada,89122-6010,02/09/2017 10:00:00 AM,'A' Grade,7-eleven #32224 - snack bar 5576 boulder hwy
1,PR0009484,venetian main kitchen warewash,3355 s las vegas blvd,Las Vegas,Nevada,89109-8941,10/24/2017 01:30:00 PM,'A' Grade,venetian main kitchen warewash 3355 s las vega...
2,PR0112115,pizza rock - cookline / pantry,2300 paseo verde pkwy,Henderson,Nevada,89052,10/04/2017 02:20:00 PM,'A' Grade,pizza rock - cookline / pantry 2300 paseo verd...
3,PR0116003,bell's bbq,10895 s eastern 130,Henderson,Nevada,89052,10/23/2017 04:30:00 PM,'B' Downgrade,bell's bbq 10895 s eastern 130
4,PR0020713,dairy queen,7501 w lake mead,Las Vegas,Nevada,89128,10/10/2017 03:15:00 PM,'A' Grade,dairy queen 7501 w lake mead
...,...,...,...,...,...,...,...,...,...
148330,PR0017255,subway #35507,6770 n durango dr,Las Vegas,Nevada,89149-4497,12/22/2016 09:30:00 AM,'A' Grade,subway #35507 6770 n durango dr
148331,PR0100603,centerplate club level restaurant remote servi...,850 n las vegas,Las Vegas,Nevada,89101,03/25/2017 11:25:00 AM,'A' Grade,centerplate club level restaurant remote servi...
148332,PR0000007,mcdonald's restaurant 31454,10590 southern highlands pkwy,Las Vegas,Nevada,89141-4373,06/02/2017 09:55:00 AM,'A' Grade,mcdonald's restaurant 31454 10590 southern hig...
148333,PR0109205,cromwell giada bar,3595 s las vegas blvd,Las Vegas,Nevada,89109-8918,05/23/2017 01:30:00 PM,'A' Grade,cromwell giada bar 3595 s las vegas blvd


In [11]:
# remove "," from columns address
ri_transformed["address"] = ri_transformed.address.str.replace(',','')

# set name & address to lower case
ri_transformed["restaurant_name"] = ri_transformed["restaurant_name"].str.lower()
ri_transformed["address"] = ri_transformed["address"].str.lower()

ri_transformed

,permit_number,restaurant_name,address,city,state_cd,zip,inspection_time,inspection_result
0,PR0003177,7-eleven #32224 - snack bar,5576 boulder hwy,Las Vegas,Nevada,89122-6010,02/09/2017 10:00:00 AM,'A' Grade
1,PR0009484,venetian main kitchen warewash,3355 s las vegas blvd,Las Vegas,Nevada,89109-8941,10/24/2017 01:30:00 PM,'A' Grade
2,PR0112115,pizza rock - cookline / pantry,2300 paseo verde pkwy,Henderson,Nevada,89052,10/04/2017 02:20:00 PM,'A' Grade
3,PR0116003,bell's bbq,10895 s eastern 130,Henderson,Nevada,89052,10/23/2017 04:30:00 PM,'B' Downgrade
4,PR0020713,dairy queen,7501 w lake mead,Las Vegas,Nevada,89128,10/10/2017 03:15:00 PM,'A' Grade
...,...,...,...,...,...,...,...,...
148330,PR0017255,subway #35507,6770 n durango dr,Las Vegas,Nevada,89149-4497,12/22/2016 09:30:00 AM,'A' Grade
148331,PR0100603,centerplate club level restaurant remote servi...,850 n las vegas,Las Vegas,Nevada,89101,03/25/2017 11:25:00 AM,'A' Grade
148332,PR0000007,mcdonald's restaurant 31454,10590 southern highlands pkwy,Las Vegas,Nevada,89141-4373,06/02/2017 09:55:00 AM,'A' Grade
148333,PR0109205,cromwell giada bar,3595 s las vegas blvd,Las Vegas,Nevada,89109-8918,05/23/2017 01:30:00 PM,'A' Grade


In [12]:
#create new df with key
ri_new_key = ri_transformed

In [13]:
#create new column in new data frame with address and name to use as key
ri_new_key["name_key"] = ri_new_key["restaurant_name"] + " " + ri_new_key["address"] 
ri_new_key.head(3)

,permit_number,restaurant_name,address,city,state_cd,zip,inspection_time,inspection_result,name_key
0,PR0003177,7-eleven #32224 - snack bar,5576 boulder hwy,Las Vegas,Nevada,89122-6010,02/09/2017 10:00:00 AM,'A' Grade,7-eleven #32224 - snack bar 5576 boulder hwy
1,PR0009484,venetian main kitchen warewash,3355 s las vegas blvd,Las Vegas,Nevada,89109-8941,10/24/2017 01:30:00 PM,'A' Grade,venetian main kitchen warewash 3355 s las vega...
2,PR0112115,pizza rock - cookline / pantry,2300 paseo verde pkwy,Henderson,Nevada,89052,10/04/2017 02:20:00 PM,'A' Grade,pizza rock - cookline / pantry 2300 paseo verd...


In [14]:
connection_string = "postgres:postgres@localhost:5432/ETLproject_db"
engine = create_engine(f'postgresql://{connection_string}')

In [15]:
engine.table_names()

['yelp', 'ri']

In [16]:
yelp_transformed.to_sql(name='yelp', con=engine, if_exists='append', index=False)

In [21]:
pd.read_sql_query('select * from yelp', con=engine).head()

,id,biz_name,address,city,state_cd,postal_code,stars,review_count,categories,name_key
0,1,bdj realty,2620 regatta dr ste 102,Las Vegas,NV,89128,4.0,5,Real Estate Services;Real Estate;Home Services...,bdj realty 2620 regatta dr ste 102
1,2,soccer zone,7240 w lake mead blvd ste 4,Las Vegas,NV,89128,1.5,9,Shopping;Sporting Goods,soccer zone 7240 w lake mead blvd ste 4
2,3,alfredo's jewelry,5775 s eastern ste 103,Las Vegas,NV,89119,4.5,23,Shopping;Jewelry;Watch Repair;Local Services,alfredo's jewelry 5775 s eastern ste 103
3,4,pampered hair passionate about hair,2470 paseo verde pkwy,Henderson,NV,89074,5.0,3,Hair Salons;Blow Dry/Out Services;Hair Stylist...,pampered hair passionate about hair 2470 paseo...
4,5,task electric,7260 cimarron rd ste 130,Las Vegas,NV,89113,4.5,20,Home Services;Lighting Fixtures & Equipment;Lo...,task electric 7260 cimarron rd ste 130


In [20]:
ri_new_key.to_sql(name='ri', con=engine, if_exists='append', index=False)

In [22]:
pd.read_sql_query('select * from ri', con=engine).head()

,id,permit_number,restaurant_name,address,city,state_cd,zip,inspection_time,inspection_result,name_key
0,1,PR0003177,7-eleven #32224 - snack bar,5576 boulder hwy,Las Vegas,Nevada,89122-6010,2017-02-09 10:00:00,'A' Grade,7-eleven #32224 - snack bar 5576 boulder hwy
1,2,PR0009484,venetian main kitchen warewash,3355 s las vegas blvd,Las Vegas,Nevada,89109-8941,2017-10-24 13:30:00,'A' Grade,venetian main kitchen warewash 3355 s las vega...
2,3,PR0112115,pizza rock - cookline / pantry,2300 paseo verde pkwy,Henderson,Nevada,89052,2017-10-04 14:20:00,'A' Grade,pizza rock - cookline / pantry 2300 paseo verd...
3,4,PR0116003,bell's bbq,10895 s eastern 130,Henderson,Nevada,89052,2017-10-23 16:30:00,'B' Downgrade,bell's bbq 10895 s eastern 130
4,5,PR0020713,dairy queen,7501 w lake mead,Las Vegas,Nevada,89128,2017-10-10 15:15:00,'A' Grade,dairy queen 7501 w lake mead
